In [1]:
# packages
import sys
import os
import math
import torch
import random
import pickle
import calendar
import numpy as np
import config
import training 
import matplotlib.pyplot as plt
import evaluation as E
from torch.utils.data import TensorDataset, DataLoader
os.environ['KMP_DUPLICATE_LIB_OK']='True'
torch.set_default_tensor_type(torch.FloatTensor)

In [2]:
datasets = os.listdir('./dataset')
datasets = [f for f in datasets if (f.startswith('Dataset') and f.endswith('.p'))]
datasets.sort()
datasets

['Dataset_acuteinflammation.p',
 'Dataset_balancescale.p',
 'Dataset_breastcancerwisc.p',
 'Dataset_cardiotocography3clases.p',
 'Dataset_energyy1.p',
 'Dataset_energyy2.p',
 'Dataset_iris.p',
 'Dataset_mammographic.p',
 'Dataset_pendigits.p',
 'Dataset_seeds.p',
 'Dataset_tictactoe.p',
 'Dataset_vertebralcolumn2clases.p',
 'Dataset_vertebralcolumn3clases.p']

In [3]:
seeds = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
lr_rts = [0.005, 0.]
train_epsilons = [0., 0.05, 0.1]
test_epsilons = [0.05, 0.1]

# Evaluation

In [4]:
results = torch.zeros([13, 10, 2, 3, 2, 2])

In [5]:
for i, dataset in enumerate(datasets):
    print(dataset)
    for j, seed in enumerate(seeds):
        for k, lr_rt in enumerate(lr_rts):
            for l, train_epsilon in enumerate(train_epsilons):
                for m, test_epsilon in enumerate(test_epsilons):
                    datapath = os.path.join(f'./dataset/{dataset}')
                    with open(datapath, 'rb') as f:
                        data = pickle.load(f)
                    X_train    = data['X_train']
                    y_train    = data['y_train']
                    X_valid    = data['X_valid']
                    y_valid    = data['y_valid']
                    X_test     = data['X_test']
                    y_test     = data['y_test']
                    data_name  = data['name']

                    setup = f'dataset:{data_name}_epsilon:{train_epsilon}_lr_rt:{lr_rt}_seed:{seed}'

                    model = torch.load(f'./report/pNN_{setup}')
                    model.SetParameter('N', config.N_test)
                    model.SetParameter('epsilon', test_epsilon)

                    acc, std = E.BASIC_variation(model, X_test, y_test)
                    results[i,j,k,l,m,0], results[i,j,k,l,m,1] = acc, std

Dataset_acuteinflammation.p
Dataset_balancescale.p
Dataset_breastcancerwisc.p
Dataset_cardiotocography3clases.p
Dataset_energyy1.p
Dataset_energyy2.p
Dataset_iris.p
Dataset_mammographic.p
Dataset_pendigits.p
Dataset_seeds.p
Dataset_tictactoe.p
Dataset_vertebralcolumn2clases.p
Dataset_vertebralcolumn3clases.p


In [6]:
# torch.save(results, f'./report/EvaluationMatrix')

In [7]:
# results = torch.load(f'./results_for_paper/EvaluationMatrix')
# results.shape

In [8]:
results_learnable = results[:,:,0,:,:,:]
results_learnable.shape

torch.Size([13, 10, 3, 2, 2])

In [9]:
results_nonlearnable = results[:,:,1,:,:,:]
results_nonlearnable.shape

torch.Size([13, 10, 3, 2, 2])

# record results

In [10]:
final_results = np.zeros([13,2,4,2])

In [11]:
def find_best(results):
    best_seed = torch.argmax(results[:,:,0], dim=1)
    temp = []
    for i, j in zip(range(13),best_seed):
        temp.append(results[i,j,:].numpy())
    return np.array(temp)

## variation + learnable

5% variation

In [12]:
final_results[:,:,0,0] = find_best(results_learnable[:, :, 1, 0, :])

10% variation

In [13]:
final_results[:,:,0,1] = find_best(results_learnable[:, :, 2, 1, :])

## nominal + learnable

5% variation

In [14]:
final_results[:,:,1,0] = find_best(results_learnable[:, :, 0, 0, :])

10% variation

In [15]:
final_results[:,:,1,1] = find_best(results_learnable[:, :, 0, 1, :])

## variation + non-learnable

5% variation

In [16]:
final_results[:,:,2,0] = find_best(results_nonlearnable[:, :, 1, 0, :])

10% variation

In [17]:
final_results[:,:,2,1] = find_best(results_nonlearnable[:, :, 2, 1, :])

## non-variation + non-learnable

5% variation

In [18]:
final_results[:,:,3,0] = find_best(results_nonlearnable[:, :, 0, 0, :])

10% variation

In [19]:
final_results[:,:,3,1] = find_best(results_nonlearnable[:, :, 0, 1, :])

# final_results = [dataset, performance, experiment, variance]
13 datasets

2 performance: mean, std 

3 experiments: both, only lnc, only variation aware, neither

2 variances: 5% 10%

## Compare ablation study

In [20]:
np.savetxt(f'./report/result_mean_5%_ACC.txt',  final_results[:,0,:,0], fmt='%.5f', delimiter='\t')
np.savetxt(f'./report/result_std_5%_ACC.txt',   final_results[:,1,:,0], fmt='%.5f', delimiter='\t')
np.savetxt(f'./report/result_mean_10%_ACC.txt', final_results[:,0,:,1], fmt='%.5f', delimiter='\t')
np.savetxt(f'./report/result_std_10%_ACC.txt',  final_results[:,1,:,1], fmt='%.5f', delimiter='\t')

In [21]:
final_results[:,0,:,0]

array([[1.        , 0.95560002, 0.89120001, 0.80000001],
       [0.87992066, 0.86984128, 0.77817458, 0.80976188],
       [0.96292859, 0.95071429, 0.92435712, 0.90735716],
       [0.7636385 , 0.76802814, 0.7727465 , 0.7751174 ],
       [0.87149352, 0.88876623, 0.83766234, 0.81558442],
       [0.88292205, 0.83681816, 0.72662336, 0.66305196],
       [0.91225809, 0.70580643, 0.80709678, 0.72645164],
       [0.78248703, 0.61316061, 0.68917096, 0.63005179],
       [0.52166438, 0.39124146, 0.36479309, 0.31270123],
       [0.75511628, 0.74906975, 0.76604652, 0.51999998],
       [0.71291667, 0.6821875 , 0.63302082, 0.63020831],
       [0.6363492 , 0.70587301, 0.68190473, 0.68095237],
       [0.61873019, 0.61238098, 0.53920633, 0.60206348]])

In [22]:
final_results[:,0,:,1]

array([[0.99879998, 0.88200003, 0.87440002, 0.82080001],
       [0.87690479, 0.82420635, 0.68230158, 0.70595241],
       [0.93049997, 0.88042855, 0.87764287, 0.86435711],
       [0.76323944, 0.6477465 , 0.76753521, 0.75645542],
       [0.84662336, 0.83454543, 0.83545452, 0.70857143],
       [0.86655843, 0.77227271, 0.7138961 , 0.61532468],
       [0.84258062, 0.61709678, 0.73290324, 0.6587097 ],
       [0.76554406, 0.56870466, 0.64176166, 0.60906738],
       [0.54774445, 0.33079126, 0.25259209, 0.23415643],
       [0.8195349 , 0.65604651, 0.71697676, 0.41302326],
       [0.65984374, 0.63020831, 0.63447917, 0.63020831],
       [0.63492066, 0.65920633, 0.65047622, 0.66555554],
       [0.63396823, 0.51396823, 0.56047618, 0.57015872]])